# 05_preprocess — 전처리란 무엇이고 왜 필요할까?

오늘의 목표
1) **인코딩**: 글자(문자열)는 모델이 이해 못 한다 → 숫자로 바꿔주기
2) **스케일링**: 값의 크기가 다르면 공정한 비교가 안 된다 → 크기 맞춰주기
3) **정규화**: 각 데이터의 길이를 똑같이 맞춰주는 방법
4) **데이터 누수**: 시험지를 미리 보면 안 된다 → 훈련 데이터로만 fit

## 인코딩이란?

- 모델은 글자(문자열)를 직접 계산할 수 없음
- 예: 색깔 = "빨강(red)", "초록(green)", "파랑(blue)"
- 글자를 그대로 넣으면 모델은 "이게 뭐지?" 하고 멈춤

-> 마치 한국어 시험지를 영어 선생님에게 가져가면, 선생님은 못 알아보는 것과 같음
그래서 "숫자"로 번역(인코딩)을 해줘야 함

In [1]:
import pandas as pd

data = pd.DataFrame({
    "color": ["red","green","blue","red","green","blue"],
    "target": [1,0,0,1,0,0]
})
data

,color,target
0,red,1
1,green,0
2,blue,0
3,red,1
4,green,0
5,blue,0


## 레이블 인코딩 vs 원-핫 인코딩

1) **레이블 인코딩(Label Encoding)**
   - red=0, green=1, blue=2 처럼 번호 붙이기
   - 문제: 0, 1, 2 이런식으로 하면 숫자의 크기가 반영, 크기로 인해서 모델이 오해(착시) 0<1<2

2) **원-핫 인코딩(One-Hot Encoding)**
   - red = [1,0,0]
   - green = [0,1,0]
   - blue = [0,0,1]
   - 순서를 만들지 않고, 단지 "있다/없다"만 표시

In [23]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# 레이블 인코딩
le = LabelEncoder()
data["color_label"] = le.fit_transform(data["color"])

# 원-핫 인코딩
ohe = OneHotEncoder(sparse_output=False)
ohe_matrix = ohe.fit_transform(data[["color"]])
ohe_df = pd.DataFrame(ohe_matrix, columns=ohe.categories_[0])

# 합치기
data_ohe = pd.concat([data, ohe_df], axis=1)
display(data_ohe)

,color,target,color_label,blue,green,red
0,red,1,2,0.0,0.0,1.0
1,green,0,1,0.0,1.0,0.0
2,blue,0,0,1.0,0.0,0.0
3,red,1,2,0.0,0.0,1.0
4,green,0,1,0.0,1.0,0.0
5,blue,0,0,1.0,0.0,0.0


- "color_label" 열은 0,1,2로 번호를 붙인 것 (순서 착각 위험)
- "blue, green, red" 열은 원-핫 인코딩 결과 (순서 없음, 안전)

👉 실제로는 **원-핫 인코딩**을 많이 씁니다.

### Ordinal(순서형) 인코딩 — LabelEncoder가 유용한 경우
- 명목형(순서 없음): 원-핫
- **순서형(작<중<대)**: 레이블/오디널 인코딩이 자연스러움

1) **레이블 인코딩(Label Encoding)**
   - red=0, green=1, blue=2 처럼 번호 붙이기
   - 문제: 0<1<2 같은 **가짜 순서**가 생겨 선형/거리 기반 모델에서 **왜곡**을 만든다.
     (명목형 범주에는 부적합. 트리 모델에서는 제한적으로 가능)

In [2]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

# 데이터 프레임 생성
df_ord = pd.DataFrame({"size":["small","medium","large","medium","small"]})
# Ordinal(순서형) 인코더 가지고 범주형 -> 숫자로 인코더
#  categories = 파라미터에 순서를 직접 우리가 지정해줌
# small 0, medium 1, large 2
ord_enc = OrdinalEncoder(categories=[["small","medium","large"]])
# fit_transform : 학습+ 변환 한번에 제가 귀찮아서 자주 애용 아이
df_ord["size_code"] = ord_enc.fit_transform(df_ord[["size"]])
display(df_ord)

,size,size_code
0,small,0.0
1,medium,1.0
2,large,2.0
3,medium,1.0
4,small,0.0


## 스케일링이란?

- 두 가지 정보가 있습니다.
  - 학생 키 = [150cm, 160cm, 170cm] → 범위: 20
  - 학생 용돈 = [1000원, 20000원, 50000원] → 범위: 49000

-> 용돈 값이 너무 커서 모델은 키를 무시하게 됩니다.
마치 수학 점수는 0~100, 음악 점수는 0~1억일 때 → 음악만 중요해지는 상황입니다.
그래서 범위를 비슷하게 맞춰주는 작업이 필요합니다.

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

X = np.array([[150,1000],
              [160,20000],
              [170,50000]])

scaler = StandardScaler()
# 평균 0, 표준편차 1 (-1 ~ 1)
# 모델이 공평하게 학습(비교도 하구)
X_scaled = scaler.fit_transform(X)

df_scale = pd.DataFrame(
    np.hstack([X, X_scaled]),
    columns=["키(cm)","용돈(원)","키(스케일링 후)","용돈(스케일링 후)"]
)
display(df_scale)

,키(cm),용돈(원),키(스케일링 후),용돈(스케일링 후)
0,150.0,1000.0,-1.224745,-1.123698
1,160.0,20000.0,0.000000,-0.181775
2,170.0,50000.0,1.224745,1.305473


- 원래 데이터는 "용돈" 값이 너무 커서 차이가 엄청 큼
- 스케일링 후에는 "키"와 "용돈"이 비슷한 범위로 맞춰짐
##### ->  이제 두 특성이 공정하게 반영될 수 있음

### RobustScaler — 이상치에 강한 스케일러
- 중앙값/사분위 범위(IQR)로 스케일링 → 평균/표준편차 대신 **강건한 통계량** 사용

In [4]:
import numpy as np, pandas as pd
from sklearn.preprocessing import RobustScaler, StandardScaler

X = np.array([[1., 10.],
              [2., 12.],
              [3., 13.],
              [100., 1000.]])   # 이상치
# 평균 = 0, 표준편차 1로 변환 (z-Score)
# 평균을 가지고 노니까 이상치에 민감
sc = StandardScaler().fit_transform(X)
# 중앙값, IQR을 가지고 스케일링
# 이상치에 상대적으로 덜 민감함
rb = RobustScaler().fit_transform(X)

df_rb = pd.DataFrame(np.hstack([X, sc, rb]),
                     columns=["x1","x2","Std_x1","Std_x2","Robust_x1","Robust_x2"])
display(df_rb)

,x1,x2,Std_x1,Std_x2,Robust_x1,Robust_x2
0,1.0,10.0,-0.600832,-0.581243,-0.058824,-0.010070
1,2.0,12.0,-0.577270,-0.576570,-0.019608,-0.002014
2,3.0,13.0,-0.553708,-0.574233,0.019608,0.002014
3,100.0,1000.0,1.731810,1.732045,3.823529,3.977845


### MinMaxScaler vs StandardScaler — KNN 성능 비교
- MinMax: [0,1] 구간으로 맞춤 (최솟값/최댓값 기반)
- Standard: 평균 0, 표준편차 1 (정규 분포 가정)
- 어떤 스케일러가 좋은지는 **데이터 분포/모델**에 따라 다르다.

In [10]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris

# iris 데이터 로드
X, y = load_iris(return_X_y=True)
# 학습데이터, 테스트 데이터 나눔
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, stratify=y, random_state=42)


pipe_minmax = Pipeline([("scaler", MinMaxScaler()), ("knn", KNeighborsClassifier(n_neighbors=5))])
pipe_std    = Pipeline([("scaler", StandardScaler()), ("knn", KNeighborsClassifier(n_neighbors=5))])

pipe_minmax.fit(X_train, y_train); pred_mm = pipe_minmax.predict(X_test)
pipe_std.fit(X_train, y_train);    pred_sd = pipe_std.predict(X_test)

print(f"[MinMax] Acc={accuracy_score(y_test, pred_mm):.3f}, F1(macro)={f1_score(y_test, pred_mm, average='macro'):.3f}")
print(f"[Std   ] Acc={accuracy_score(y_test, pred_sd):.3f}, F1(macro)={f1_score(y_test, pred_sd, average='macro'):.3f}")

[MinMax] Acc=0.933, F1(macro)=0.933
[Std   ] Acc=0.911, F1(macro)=0.910


## 정규화란?

정규화는 각 데이터 벡터의 길이를 1로 맞추는 것.
예: [3,4] → 길이=5 → [0.6,0.8]

-> 마치 지도에서 "집까지의 거리"는 무시하고, "집이 어느 방향에 있는지"만 보는 것과 같음

음악 취향 비유:
- A는 록 100곡+재즈 50곡, B는 록 10곡+재즈 5곡
- → 절대 개수는 다르지만, 비율(방향)은 똑같음
- → 정규화하면 같은 취향 패턴으로 인식됨

In [11]:
from sklearn.preprocessing import Normalizer

X = np.array([[3,4],
              [1,2],
              [10,0]])

normalizer = Normalizer()
X_norm = normalizer.fit_transform(X)

df_norm = pd.DataFrame(
    np.hstack([X, X_norm]),
    columns=["x1","x2","x1(정규화 후)","x2(정규화 후)"]
)
display(df_norm)

,x1,x2,x1(정규화 후),x2(정규화 후)
0,3.0,4.0,0.600000,0.800000
1,1.0,2.0,0.447214,0.894427
2,10.0,0.0,1.000000,0.000000


### 정규화와 코사인 유사도
- 정규화하면 벡터의 길이가 1이 되어 **코사인 유사도 = 내적**이 됨.
- LLM 임베딩(단어/문장 비교)은 보통 코사인 유사도를 사용 → 정규화와 직접 연결.

In [12]:
import numpy as np
from sklearn.preprocessing import Normalizer
from numpy.linalg import norm

A = np.array([[3.,4.]])
B = np.array([[6.,8.]])

normer = Normalizer()
A_n = normer.fit_transform(A); B_n = normer.transform(B)

cos = (A @ B.T) / (norm(A) * norm(B))
dot_after_norm = (A_n @ B_n.T)
print(f"cosine(A,B)= {cos.item()}")
print(f"dot(normalized)= {dot_after_norm.item()}")

cosine(A,B)= 1.0
dot(normalized)= 1.0


## 데이터 누수

- 훈련+시험 전체 데이터를 fit → 시험지 평균/표준편차까지 미리 알게 됨
- 마치 학생이 **시험지를 미리 훔쳐보고 공부한 것**과 같습니다.

-> 시험 볼 때는 성적이 뻥튀기되지만, 실제 실력은 아닙니다.
Pipeline을 쓰면 "훈련 데이터로만 fit, 시험 데이터는 transform" 규칙을 자동으로 지켜 안전합니다.

In [13]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

# (나쁜 예시) 전체 데이터로 fit
# 스탠다드스케일러(평균, 표준편차 계산)
# 험 데이터(X_test) 데이터도 평균/표준편차가 반영이 됨
# 데이터 누수
scaler_bad = StandardScaler().fit(X)
# 위에서 계산된(시험 데이터가 포함된) 평균과 표준편차로, 학습/시험 데이터를 변환
X_train_bad = scaler_bad.transform(X_train)
X_test_bad  = scaler_bad.transform(X_test)
knn_bad = KNeighborsClassifier(n_neighbors=5).fit(X_train_bad, y_train)
print("누수 발생 Acc:", knn_bad.score(X_test_bad, y_test))



누수 발생 Acc: 1.0


In [14]:
# (Pipeline) 훈련에만 fit
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier(n_neighbors=5))
])
pipe.fit(X_train, y_train)
print("Pipeline Acc:", pipe.score(X_test, y_test))

Pipeline Acc: 1.0


## 요약

1) 인코딩: 글자는 숫자로 바꿔야 한다 → 원-핫 인코딩이 안전
2) 스케일링: 값 크기 차이가 크면 왜곡된다 → StandardScaler, MinMaxScaler 사용
3) 정규화: 각 샘플 길이=1 → 방향만 중요할 때 유용
4) 데이터 누수: 시험지 정보가 훈련에 들어가면 안 된다 → Pipeline으로 안전하게 처리

## 실습 미션

1) 색상 데이터에 LabelEncoder만 쓰면 어떤 문제가 생기는지 다시 확인
2) 위의 키/용돈 데이터에 MinMaxScaler를 적용해 보세요. 결과를 StandardScaler와 비교
3) 정규화를 하지 않았을 때와 했을 때의 차이를 그림으로 확인
4) (도전) Pipeline에 다른 모델(KNN 대신 로지스틱)을 넣어서 동작을 확인

## 데이터 누수(Leakage) — Iris 데이터로 다시 확인하기

이번에는 Iris 데이터(붓꽃)로 “나쁜 전처리 vs 올바른 전처리(Pipeline)”를 직접 비교

핵심 메시지
- 올바른 방법: 전처리는 **훈련셋에만 fit**, 시험셋에는 **transform만** 적용
- 쉬운 방법: **Pipeline**을 사용하면 이 규칙이 자동으로 지켜진다

In [15]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
X_full, y_full = iris.data, iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X_full, y_full, test_size=0.3, stratify=y_full, random_state=42
)

X_train.shape, X_test.shape

((105, 4), (45, 4))

### 나쁜 전처리(누수) 방법
- 전체 데이터(X_train + X_test)를 합쳐서 스케일러(StandardScaler)에 **fit**을 해버리면,
- 시험 데이터의 평균/표준편차 정보까지 학습 단계에 스며들게 됩니다.
- 이렇게 하면 **성능이 실제보다 과장**될 수 있습니다. (공정하지 않음)

In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score

# (나쁜 예시) 전체 데이터로 스케일러 fit
scaler_bad = StandardScaler().fit(X_full)            # 금지: X_train + X_test를 함께 사용
X_train_bad = scaler_bad.transform(X_train)
X_test_bad  = scaler_bad.transform(X_test)

# KNN 학습/평가
knn_bad = KNeighborsClassifier(n_neighbors=5)
knn_bad.fit(X_train_bad, y_train)
pred_bad = knn_bad.predict(X_test_bad)

print(f"[누수 발생·KNN] Acc={accuracy_score(y_test, pred_bad):.3f}, F1(macro)={f1_score(y_test, pred_bad, average='macro'):.3f}")

[누수 발생·KNN] Acc=0.911, F1(macro)=0.910


### 올바른 전처리 방법: Pipeline
- 훈련셋에만 전처리 `fit` → 시험셋에는 `transform`만 적용
- ColumnTransformer/Pipeline을 쓰면 이 규칙이 자동으로 지켜집니다

In [17]:
from sklearn.pipeline import Pipeline

# 아 귀찮아 파이프라인써야지(스케일링하고, 모델선정 한번에)
pipe_knn = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier(n_neighbors=5))
])
# 데이터 누수를 방지하기 위해서 훈련 데이터 셋에만 fit(훈련)
# 테스트(시험) 데이터 셋은 한 번도 스케일링 또는 한번도 보지 못했다구 -> 누수가 없다구
pipe_knn.fit(X_train, y_train)
pred_pipe = pipe_knn.predict(X_test)

print(f"[Pipeline·KNN]  Acc={accuracy_score(y_test, pred_pipe):.3f}, F1(macro)={f1_score(y_test, pred_pipe, average='macro'):.3f}")

[Pipeline·KNN]  Acc=0.911, F1(macro)=0.910


### 추가 확인: 로지스틱 회귀(선형 모델)에서도 확인해 보기
- 선형 모델은 스케일링의 영향을 더 많이 받는 편입니다.
- “누수 전처리 vs Pipeline”을 로지스틱 회귀에서도 비교합니다.

In [18]:
from sklearn.linear_model import LogisticRegression

# (나쁜 예시) 전체 데이터로 fit한 스케일러 사용
lr_bad = LogisticRegression(max_iter=5000, random_state=42)
lr_bad.fit(X_train_bad, y_train)
pred_lr_bad = lr_bad.predict(X_test_bad)

# (올바른 예시) Pipeline
pipe_lr = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(max_iter=5000, random_state=42))
])
pipe_lr.fit(X_train, y_train)
pred_lr_pipe = pipe_lr.predict(X_test)

print(f"[누수 발생·LR]  Acc={accuracy_score(y_test, pred_lr_bad):.3f}, F1(macro)={f1_score(y_test, pred_lr_bad, average='macro'):.3f}")
print(f"[Pipeline·LR]   Acc={accuracy_score(y_test, pred_lr_pipe):.3f}, F1(macro)={f1_score(y_test, pred_lr_pipe, average='macro'):.3f}")

[누수 발생·LR]  Acc=0.911, F1(macro)=0.911
[Pipeline·LR]   Acc=0.911, F1(macro)=0.911


## 결과 해석

- 데이터가 단순한 경우에는 차이가 작게 보일 수 있지만,
  **누수 전처리 방식은 원칙적으로 금지**입니다.
- 실제 업무 데이터(복잡, 잡음, 범주형 많음)에서는 누수 전처리를 하면
  성능이 **과장되어 보이고**, 배포 후 성능 하락으로 이어질 가능성이 큽니다.
- **Pipeline**을 쓰면 전처리 `fit`이 훈련셋에만 적용되고,
  시험셋에는 `transform`만 적용되므로 **항상 안전하고 공정**합니다.

핵심 요약
- 전처리는 반드시 **훈련셋으로만 fit**
- 시험셋은 반드시 **처음 보는 데이터**처럼 다뤄야 함
- ColumnTransformer/Pipeline을 습관처럼 사용

## 교차검증으로 누수 vs Pipeline 비교

- train/test split은 한 번만 나누기 때문에 운에 따라 결과가 달라질 수 있음
- 교차검증(K-Fold CV)은 데이터를 여러 번 나눠 평균 성능을 구하기 때문에 훨씬 안정적
- 이번에는 **누수 전처리 방식**과 **Pipeline 방식**을 교차검증으로 비교해봅시다

In [19]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# (나쁜 예시) 전체 데이터로 fit → transform 적용
scaler_bad = StandardScaler().fit(X)   # 전체 데이터 사용 → 누수
X_scaled_bad = scaler_bad.transform(X)

knn_bad = KNeighborsClassifier(n_neighbors=5)
scores_bad = cross_val_score(knn_bad, X_scaled_bad, y, cv=cv, scoring="accuracy")

# (올바른 예시) Pipeline (훈련셋만 fit, 시험셋은 transform만)
pipe_knn = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier(n_neighbors=5))
])
scores_pipe = cross_val_score(pipe_knn, X, y, cv=cv, scoring="accuracy")

print("[누수 발생·KNN]   mean=%.3f ± %.3f" % (scores_bad.mean(), scores_bad.std()))
print("[Pipeline·KNN]    mean=%.3f ± %.3f" % (scores_pipe.mean(), scores_pipe.std()))

[누수 발생·KNN]   mean=0.967 ± 0.037
[Pipeline·KNN]    mean=0.973 ± 0.025


### 로지스틱 회귀(Logistic Regression)로도 비교해보기

- 로지스틱은 스케일링의 영향을 더 많이 받는 모델입니다.
- 이번에는 로지스틱 회귀에서 **누수 vs Pipeline**을 교차검증으로 비교합니다.

In [20]:
from sklearn.linear_model import LogisticRegression

# (나쁜 예시) 누수 스케일링
lr_bad = LogisticRegression(max_iter=5000, random_state=42)
scores_lr_bad = cross_val_score(lr_bad, X_scaled_bad, y, cv=cv, scoring="accuracy")

# (올바른 예시) Pipeline
pipe_lr = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(max_iter=5000, random_state=42))
])
scores_lr_pipe = cross_val_score(pipe_lr, X, y, cv=cv, scoring="accuracy")

print("[누수 발생·LR]   mean=%.3f ± %.3f" % (scores_lr_bad.mean(), scores_lr_bad.std()))
print("[Pipeline·LR]    mean=%.3f ± %.3f" % (scores_lr_pipe.mean(), scores_lr_pipe.std()))

[누수 발생·LR]   mean=0.953 ± 0.045
[Pipeline·LR]    mean=0.953 ± 0.045


## 교차검증 결과 해석

- **누수 전처리 방식**: 시험 데이터 정보를 슬쩍 본 상태라 성능이 더 높게 나올 수 있음
- **Pipeline 방식**: 훈련 데이터에만 fit → 시험 데이터에는 transform만 → 더 공정하고 현실적인 성능

핵심 정리
- 교차검증을 쓰면 “누수 vs Pipeline”의 차이를 평균 성능으로 안정적으로 비교할 수 있다
- 실제 데이터에서는 누수 방식이 **성능을 뻥튀기**해서 보여주는 경우가 많음
- **Pipeline**을 습관처럼 사용해야 안전하다